<a href="https://colab.research.google.com/github/Pmilivojevic/Bioinformatics/blob/main/bioinformatics_chapter2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import time
import copy
import random
from google.colab import drive
drive.mount('/content/drive/')
nucleotides= ['A', 'C', 'G', 'T']

Mounted at /content/drive/


In [ ]:
def symbolToNum(c):
  if c == 'A':
    return 0
  elif c == 'C':
    return 1
  elif c == 'G':
    return 2
  else:
    return 3
def patternToNum(pattern):
  if len(pattern) == 0:
    return 0
  else:
    symbol= pattern[len(pattern)-1]
    prefix=  pattern[:len(pattern)-1]
    return 4*patternToNum(prefix) + symbolToNum(symbol)
def numToSymbol(n):
  if n == 0:
    return 'A'
  elif n == 1:
    return 'C'
  elif n == 2:
    return 'G'
  else:
    return 'T'
def numToPattern(index, k):
  if k == 1:
    return numToSymbol(index)
  else:
    prefixInd= int(index/4)
    remainder= index%4
    symbol= numToSymbol(remainder)
    return numToPattern(prefixInd, k-1) + symbol
def immediateNeighbors(pattern):
  neighborhood= [pattern]

  for i in range(len(pattern)):
    symbol= pattern[i]

    for X in nucleotides:
      if X != pattern[i]:
        sample= pattern[0:i]+X+pattern[i+1:len(pattern)]
        neighborhood.append(sample)

  return neighborhood
def sufix(text):
  return text[1:len(text)]
def neighbors(pattern, d):
  if d == 0:
    return pattern

  if len(pattern) == 1:
    return nucleotides

  neighborhood= []
  sufixNeighbors= neighbors(sufix(pattern), d)

  for s in sufixNeighbors:
    if hammingDistance(s, sufix(pattern)) < d:
      for X in nucleotides:
        neighborhood.append(X+s)
    else:
      neighborhood.append(pattern[0]+s)

  return neighborhood

In [ ]:
def motifEnumeration(DNA, k, d):
  patterns= []

  for seq in DNA:
    for i in range(len(seq)-k+1):
      for pattern in neighbors(seq[i:i+k], d):
        for string in DNA:
          exist= False

          for j in range(len(string)-k+1):
            if hammingDistance(pattern, string[j:j+k]) <= d:
              exist= True
              break
          if not exist:
            break

        if exist:
         # print(pattern)
          ex= False

          for ptt in patterns:
            if pattern == ptt:
              ex= True
              break
          if not ex:
            patterns.append(pattern)

  return patterns

In [ ]:
def hammingDistance(p, q):
  d= 0
  for i in range(len(p)):
    if p[i] != q[i]:
      d+= 1

  return d

In [ ]:
def d(pattern, seq):
  minDist= hammingDistance(pattern, seq[0:len(pattern)])

  for i in range(1, len(seq)-len(pattern)+1):
    if hammingDistance(pattern, seq[i:i+len(pattern)]) < minDist:
      minDist= hammingDistance(pattern, seq[i:i+len(pattern)])

  return minDist

def dDNA(pattern, DNA):
  minDistSum= 0

  for dna in DNA:
    minDistSum+= d(pattern, dna)

  return minDistSum

In [ ]:
seq= 'TGAACTTTACCTCGATGAGGCCTTGTCGGTCTTCGAGGACTTGCTCGGCCGGCCCCCGCAGCGCTGCTTTCGTGAGTGTCAAAGCTGTTGAGACGGAGGTTGAAT AGGTGTTCTGGCCCACCAACCGATCTATATGTGGGCCTGTCGGCACCGATGGTCCTTACCCTGTCCTGTAGGCAATTTGACATAAAGTCGACGATTGGCCACCAG CTAGGGGTATTGAAACAAAGCGTACTCATTTTGATGGCCCTCAAAGGAATTATGTGACCCGCAGCTAATAGCATTGGGGGAAAGTACTTATTGGGCTGCGTGAGG TGGAAATCTCTTTCTAAGCTCGTTCTTCTCATATACGGATATTCAAGGGACACGGTTGTCGCGTTATCCAATGTATCTCGGTACGAGTGCCGTGCGCCCCATGCC TAGGGGCAATGCTCCCTGAACTGGCTAACAAGTTTGCGCCCTAGCGGACGCCAAGCAACCATGCGGATGTCTAGAGAGACTTTAGTTGCCTCAGACGTTTTAGTA CTGCCTTCCTCATTCCAAGATTGGTAACACACTGGCCAATAACAGGAATGGTAGCATACAGCGCACTATCTATACACCTTCCTACGTCTTACCAGATAGGTTTAA CAGTCTATTCATTTTACCCTCAATCGCATTGGGTTGATCATGGGGATACGCGTACGCCACCGCTACAGCATTGTCCCATGCTTTCAATGAGGGTCAGCTAACCAA CAACCTCTTAAACGAGGAACCATGGGACCGTGTCTGCGGCTCTGCCACAAACTATGCGAAGTTCGAGGCGATTCCTCCCGTCATGCATCCTAGCTATGGCCTGGT CTATTCGATGTGTCACTTTCTTTGCTCCAGTCTGTTTCGCACCGATCTTTTTCAACGACTACATGCACGTTTTGTGTGCTTCGTACTAAGCACCGACTTATAGTT TTTACTTTCCGGACGACTCAGATTCCTAGACAGCTCATACGATGCTAGATCTTTGTGATCCAATAAATGAGCGGACTGCATCTAGGGCAAGCTATTACATGATTT TTGCGGCATACATTGCGACAGGAGACCCTAGTGCCCAAACTAATAACGGTACTACCGCATGCCTACTCACTCCGATATCATGCTTCATCAAGCGTCTATCACGGG TGCGTGGTAAGCGATCCTAAGTATTTCAATGATCGTAAAGGGTGTTTTATTGCTGGGAAAAACTGACACTATAGTAGATTGAACCCGGGGGGGATTCACCGTACA ACGAAGCGTCTTAATGCGGGAAGTGGGGGCAACAACGAGGGTACTGAAATCTAAAGTTCCTATGATATCGGTATTAGCACAGTTTACGATCATCGCGCGGGTTTC ACGCGAACACTAAAAGGGCCTCGCGCGTTCCGCAGCTGCTACGAGGCAGCTCGCCGAGGTCACGCCATTCCATTCTTCAGTTAATTGTTCCTCCCCTATTCTTCA TAGCGGGTCTTACCGTCGAAATTTCTATCCGGGCCGAAGTTTAGGCAGAGGTATGGTGAGTGACGTGGAGCCATCGCTACAAGATTATCAGTTAGTCCATTCGCA ACTATCAATATCCTTTATCGAAGGTAACCAAGGCAGCAGAACCGACGCGGTCATATACCCCAATTCCCGTGGTATCTTGTAATTGGCAATTTCCCTCTCTCTCCG CAGCGATCGCCATTTTTACAGGCAAAGAGGGCGTATATCCACGATTCACCCCGCTTAATCACCGGAGCTGCTTTCTGTAAGCGGGTACGGTCCTTGTCTTCCCCT AACAGCGCATTAAGCATGATCGACGAAGTTGTGATTGCAAACATAGCACGTCATTTAGAATTCAATCCGTTAGAATTATGTAAGCAGAGGTATTCAAGGTGTCGC GCTGGTAGTAAAAGGGCGAAAAGGCCACAGCAGTCCACCCAGGTGCGTGGAATTGCGACTGGGATAAGAGCTTTAGCCTCGCTGTATGCGACGCTAGCTAAATCC TAAAGTCGCGAAGTCGCAGCTAGGATAGCGACCCGTTGCAGATAGTTTATCCTCCGAGACCTCTGTAAGATACTAATCCCTATCGCTAGTGCTCCGACATTGGCT AGCCTTTGGAAGGACCAGTGTAGACATAGTCCATGTCCACCCAGCCCGAGGCTCCCTGCAGAACCATCGTGGGCCAGGATGAGCCTACGGCGGTCGGTCTGCTAG TCTTATATGCGTGTCAAGGGGTGCACGATTAAGTGATCAGGGTGAATCGGGATACCTGATGATGGTTAAAAGCCAACTCGTCACAAGTTAATGTTAGGGACCAGA GAGACAGTCATCTGTGAGACATACATCGAACCTGATTGAAAGGTGTATGAATTCTTGGTTTGCTCCGTAGAGCAGACGTGCTTAGCTAGCGAATGATTACCTCCC CGTTGAATACCCTCCTTTCTTTACCAACCCGACGAGAAAGCACCATACCCGATGAATCAGGTTGATATGTGTACACGCGTCAAATGAAAACGGGGAATCCCTAAG GTGTGTATCTTGCTACTACTCATGATTCCAACCCTGACCTCTTATCGTATCGCAAGCCCATGCATAGATCTGCGTGAGCGACCTACGCTCCGCCTACTGTTCTAG CCAATGCCATTTAATTGCTGACCTCTACAGCCGGGGGTAAAGGCACGCACGCCCGCACTCTTGTCTCTCCAAGCGTAGGTACGCATACTAAGGATACCTTGTTAC TCCAGCCGTGTCCTGGGCCCGGGCGTAAAAAAAGCAAGATTGTACTTAGTATGGTACATATGCGCGCGAGGCCTACCAAGCCGTGGGGAGCTATCATAGCCCTAG ACATTGAGAAGGGCTTCACCAGATTCGGGATAGGGAAGGAGTAGCGCCGCCCTTTCACCGAGCAAAAAGGGAGAGTATAATTCGCAAGTCCTTGAGCGTGGCTGA TGGCCTAGCTGAATATTATATTGTGTTATCAGAAGCCATACAAGCCCGTGAACAGTGGCTCTCTGAATCCGCGGATCTGAACGGGGGATTTTTGCCGTGTTTAAC TTTCATCTGTATCGGTCGACTATCCAACGCACAGTAATCATTACCGCCATTTGGTCACAGGGCAACGTATCTGATAGATTAGCACAACTAAGCTGGTGTGTCATT GGGTGAGATTTTACGGGGCGTTGAGTCTGAAGATTTAAACGGACTCTATTGCAGTAGAGTCTAGTCAGGGGATGTTTGAATGAGACGATTGGACTAACGCGGTGA CTCAGAATGGCGTTAGCCCCACGTGAGGCTGCGTATAACCGGAGAAGATCGTGCGCGCCGCGATTGTACCGGCTGGTTTTGATTTTGAATAAGTCGTACTAGATG GCATCGGTCAATGGCACCAACAGCCCGGCTATCAGAGACGGAAAGCCTCCAGAACTTGGACGAAGGCCGATTCGGTAATGCACCAAACGGCGTCCGCGCATGGCA AGCAATCTTCGTTCGCAAGGACTACCCGATACTAGGGTCTAGAACTGTAGCATTATCAGGCTCGAGACGCTTAGATCACACAGCTGCTAGTGACCAGCTCTCGTC ACGAACAGGCCCACCTCCCTCTATTACGAGAGAGTCGACCACTGCTTCACTTCTGGCACGCGGGTCGTGCCAGGGGTATTTGAAACATACTACGCATGCACTAAG CTCGGGTTAGGTTCAAGCAGGGACTCGGGGCATAGACCATACCATCGGAGACGTAGATTGGTATATAAGCTACACGAGGGTTAGACGAAAGAACAACAGATCAAA TCGCAATCGTTTCATCCCGATTTACTTTTCCGTATATCAGTGATCCGCATTAGCGACGGGGACGGTGCATCGTGGGTTCCGACTCGTCCAGATGCATGCTATCGG GGACTGGAACAAAGCTTACTGGGTCTGAAGCAGCACTTTTTTCACAACTGTGCCTGGGCTAAGTGTGACACGCCTGGAGGGTGCTTTATATGGTCATTTAGAAGT'
DNA= []
string= ''
for c in seq:
  if c != ' ':
    string+= c
  else:
    DNA.append(string)
    string= ''
DNA.append(string)

pattern= 'TTGTTC'

print(dDNA(pattern, DNA))

62


In [ ]:
def medianString(DNA, k):
  patterns= []
  for i in range(1,4**k):
    patterns.append(numToPattern(i,k))

  minDist= dDNA(numToPattern(0,k), DNA)
  madian= numToPattern(0,k)

  for pattern in patterns:
    if dDNA(pattern, DNA) < minDist:
      minDist= dDNA(pattern, DNA)
      median= pattern

  return median

In [ ]:
def profile(seqs):
  if len(seqs[0]) == 1:
    matProf= np.ones((4, len(seqs)))
    for i in range(len(seqs)):
      if seqs[i] == 'A':
        matProf[0][i]+= 1
      elif seqs[i] == 'C':
        matProf[1][i]+= 1
      elif seqs[i] == 'G':
        matProf[2][i]+= 1
      else:
        matProf[3][i]+= 1
    matProf/= 5
  else:
    matProf= np.zeros((4, len(seqs[0])))

    for i in range(len(seqs[0])):
      A= 0
      C= 0
      G= 0
      T= 0
      for j in range(len(seqs)):
        if seqs[j][i] == 'A':
          A+= 1
        elif seqs[j][i] == 'C':
          C+= 1
        elif seqs[j][i] == 'G':
          G+= 1
        else:
          T+= 1

      matProf[0][i]= (A+1)/(len(seqs)+4)
      matProf[1][i]= (C+1)/(len(seqs)+4)
      matProf[2][i]= (G+1)/(len(seqs)+4)
      matProf[3][i]= (T+1)/(len(seqs)+4)

      '''matProf[0][i]= A/len(seqs)
      matProf[1][i]= C/len(seqs)
      matProf[2][i]= G/len(seqs)
      matProf[3][i]= T/len(seqs)'''

  return matProf

def consensus(prof):
  consensusStr= ''

  for i in range(len(prof[0])):
    for j in range(4):
      if prof[j][i] == max([prof[0][i], prof[1][i], prof[2][i], prof[3][i]]):
        if j == 0:
          consensusStr+= 'A'
          break
        elif j == 1:
          consensusStr+= 'C'
          break
        elif j == 2:
          consensusStr+= 'G'
          break
        elif j == 3:
          consensusStr+= 'T'
          break

  return consensusStr

def score(motifs, conKmer):
  sc= 0
  for i in range(len(motifs)):
    for j in range(len(motifs[0])):
      if motifs[i][j] != conKmer[j]:
        sc+= 1

  return sc

def motifPr(profMat, kmer):
  if kmer[0] == 'A':
    pr= profMat[0][0]
  elif kmer[0] == 'C':
    pr= profMat[1][0]
  elif kmer[0] == 'G':
    pr= profMat[2][0]
  else:
    pr= profMat[3][0]

  for i in range(1, len(kmer)):
    if kmer[i] == 'A':
      pr*= profMat[0][i]
    elif kmer[i] == 'C':
      pr*= profMat[1][i]
    elif kmer[i] == 'G':
      pr*= profMat[2][i]
    elif kmer[i] == 'T':
      pr*= profMat[3][i]

  return pr

def mostPrMotif(profMat, seq):
  k= len(profMat[0])
  maxPr= motifPr(profMat, seq[0:k])
  prMotif= seq[0:k]

  for i in range(1, len(seq)-k+1):
    if motifPr(profMat, seq[i:i+k]) > maxPr:
      maxPr= motifPr(profMat, seq[i:i+k])
      prMotif= seq[i:i+k]

  return prMotif

def profRandMotif(profMat, seq):
  k= len(profMat[0])
  pr= []
  kmers= []
  for i in range(len(seq)-k+1):
    pr.append(motifPr(profMat, seq[i:i+k]))
    kmers.append(seq[i:i+k])

  pr= pr/sum(pr)

  return random.choices(kmers, weights= pr)[0]

In [ ]:
def makeMotifs(profMat, DNA):
  motifs= []

  for seq in DNA:
    motifs.append(mostPrMotif(profMat, seq))

  return motifs

def randomizedMotifSearch(DNA, k):
  motifs= []
  for seq in DNA:
    i= random.randint(0, len(seq)-k)
    motifs.append(seq[i:i+k])

  bestMotifs= motifs

  while True:
    m= profile(motifs)
    motifs= makeMotifs(m, DNA)
    s= score(bestMotifs, consensus(m))
    if score(motifs, consensus(m)) < s:
      bestMotifs= motifs
    else:
      return bestMotifs, s

def gibsSampler(DNA, k, n):
  motifs= []
  for seq in DNA:
    i= random.randint(0, len(seq)-k)
    motifs.append(seq[i:i+k])

  bestMotifs= motifs

  for j in range(n):
    l= random.randint(0, len(DNA)-1)

    motifsNew= []
    for i in range(len(motifs)):
      if i != l:
        motifsNew.append(motifs[i])

    m= profile(motifsNew)
    motifs[l]= profRandMotif(m, DNA[l])

    if score(motifs, consensus(m)) < score(bestMotifs, consensus(m)):
      bestMotifs= motifs

  return bestMotifs

In [ ]:
def greedyMotifSearch(DNA, k):
  bestMotifs= []

  for i in range(len(DNA)):
    bestMotifs.append(DNA[i][0:k])

  motifs= []

  for i in range(len(DNA[0])-k+1):
    motifs.append(DNA[0][i:i+k])

    for i in range(1,len(DNA)):
      m= profile(motifs)

      motifs.append(mostPrMotif(m, DNA[i]))
    print('B', bestMotifs)
    print(score(bestMotifs, consensus(profile(bestMotifs))))
    print('M', motifs)
    print(score(motifs, consensus(profile(motifs))))
    if score(motifs, consensus(profile(motifs))) < score(bestMotifs, consensus(profile(bestMotifs))):
      bestMotifs= motifs
    motifs= []

  return bestMotifs

In [ ]:
dna1= []
dna1.append('TTAC')
m= profile(dna1)
dna= 'GATGTCTGTC'
print(m)
print('mostPrMotif: ', mostPrMotif(m, dna))
print('-------------------')
for i in range(10):
  print('profRandMotif: ', profRandMotif(m, dna))

[[0.2 0.2 0.4 0.2]
 [0.2 0.2 0.2 0.4]
 [0.2 0.2 0.2 0.2]
 [0.4 0.4 0.2 0.2]]
mostPrMotif:  TGTC
-------------------
profRandMotif:  TCTG
profRandMotif:  TGTC
profRandMotif:  TCTG
profRandMotif:  TGTC
profRandMotif:  TCTG
profRandMotif:  TGTC
profRandMotif:  TGTC
profRandMotif:  TCTG
profRandMotif:  TGTC
profRandMotif:  TGTC


In [ ]:
def Count(Motifs):
    count = {} # initializing the count dictionary
    k = len(Motifs[0])
    for symbol in 'ACGT':
        count[symbol] = []
        for j in range(k):
             count[symbol].append(0)

    t = len(Motifs)
    for i in range(t):
        for j in range(k):
            symbol = Motifs[i][j]
            count[symbol][j] += 1

    return count

def Profile(Motifs):
    t = len(Motifs)
    k = len(Motifs[0])

    counts= Count(Motifs)
    profile = {'A': list(np.array(counts['A'])/t), 'C': list(np.array(counts['C'])/t), 'G': list(np.array(counts['G'])/t), 'T': list(np.array(counts['T'])/t)}

    return profile

def Consensus(Motifs):
    k = len(Motifs[0])
    count = Count(Motifs)
    consensus = ""
    for j in range(k):
        m = 0
        frequentSymbol = ""
        for symbol in "ACGT":
            if count[symbol][j] > m:
                m = count[symbol][j]
                frequentSymbol = symbol
        consensus += frequentSymbol

    return consensus

def Score(Motifs):
    sc= 0
    conKmer= Consensus(Motifs)
    for i in range(len(Motifs)):
        for j in range(len(Motifs[0])):
            if Motifs[i][j] != conKmer[j]:
                sc+= 1

    return sc

def Pr(Text, Profile):
    pr= 1

    for i in range(len(Text)):
        if Text[i] == 'A':
            pr*= Profile['A'][i]
        elif Text[i] == 'C':
            pr*= Profile['C'][i]
        elif Text[i] == 'G':
            pr*= Profile['G'][i]
        else:
            pr*= Profile['T'][i]
    return pr

def ProfileMostProbableKmer(text, k, profile):
    maxPr= Pr(text[0:k], profile)
    prMotif= text[0:k]

    for i in range(1, len(text)-k+1):
        if Pr(text[i:i+k], profile) > maxPr:
            maxPr= Pr(text[i:i+k], profile)
            prMotif= text[i:i+k]

    return prMotif

def GreedyMotifSearch(Dna, k, t):
    BestMotifs = [] # output variable
    # your code here
    for i in range(t):
        BestMotifs.append(Dna[i][0:k])

    motifs= []

    for i in range(len(Dna[0])-k+1):
        motifs.append(Dna[0][i:i+k])

        for i in range(1,t):
            m= Profile(motifs)
            motifs.append(ProfileMostProbableKmer(Dna[i], k, m))

        if Score(motifs) < Score(BestMotifs):
            BestMotifs= motifs
        motifs= []

    return BestMotifs

def CountWithPseudocounts(Motifs):
    t = len(Motifs)
    k = len(Motifs[0])
    # your code here
    A= []
    C= []
    G= []
    T= []
    for i in range(k):
        countA= 1
        countC= 1
        countG= 1
        countT= 1
        for j in range(t):
            if Motifs[j][i] == 'A':
                countA+= 1
            elif Motifs[j][i] == 'C':
                countC+= 1
            elif Motifs[j][i] == 'G':
                countG+= 1
            else:
                countT+= 1
        A.append(countA)
        C.append(countC)
        G.append(countG)
        T.append(countT)

    count = {'A': A, 'C': C, 'G': G, 'T': T} # output variable
    return count

def ProfileWithPseudocounts(Motifs):
    t = len(Motifs)
    k = len(Motifs[0])

    counts= CountWithPseudocounts(Motifs)
    A= list(np.array(counts['A'])/(t+4))
    C= list(np.array(counts['C'])/(t+4))
    G= list(np.array(counts['G'])/(t+4))
    profile = {'A': A, 'C': C, 'G': G, 'T': list(np.array(counts['T'])/(t+4))} # output variable
    return profile

def MotifPr(profMat, kmer):
    pr= 1

    for i in range(len(kmer)):
        if kmer[i] == 'A':
            pr*= profMat['A'][i]
        elif kmer[i] == 'C':
            pr*= profMat['C'][i]
        elif kmer[i] == 'G':
            pr*= profMat['G'][i]
        else:
            pr*= profMat['T'][i]
    return pr

def ProfileMostProbableKmer(text, k, profile):
    maxPr= Pr(text[0:k], profile)
    prMotif= text[0:k]

    for i in range(1, len(text)-k+1):
        if Pr(text[i:i+k], profile) > maxPr:
            maxPr= Pr(text[i:i+k], profile)
            prMotif= text[i:i+k]

    return prMotif

def Motifs(Profile, Dna):
    # insert your code here
    motifs= []

    for seq in Dna:
        motifs.append(MostPrMotif(Profile, seq))

    return motifs

def Consensus(prof):
    consensusStr= ''

    for i in range(len(prof['A'])):
        Max= max([prof['A'][i], prof['C'][i], prof['G'][i], prof['T'][i]])
        if prof['A'][i] == Max:
            consensusStr+= 'A'
        elif prof['C'][i] == Max:
            consensusStr+= 'C'
        elif prof['G'][i] == Max:
            consensusStr+= 'G'
        elif prof['T'][i] == Max:
            consensusStr+= 'T'

    return consensusStr

def Score(motifs):
    sc= 0
    conKmer= Consensus(ProfileWithPseudocounts(motifs))
    for i in range(len(motifs)):
        for j in range(len(motifs[0])):
            if motifs[i][j] != conKmer[j]:
                sc+= 1

    return sc

def GreedyMotifSearchWithPseudocounts(Dna, k, t):
    BestMotifs = [] # output variable
    # your code here
    for i in range(t):
        BestMotifs.append(Dna[i][0:k])

    motifs= []

    for i in range(len(Dna[0])-k+1):
        motifs.append(Dna[0][i:i+k])

        for i in range(1,t):
            m= ProfileWithPseudocounts(motifs)
            motifs.append(MostPrMotif(m, Dna[i]))

        if Score(motifs) < Score(BestMotifs):
            BestMotifs= motifs
        motifs= []

    return BestMotifs

def RandomMotifs(Dna, k, t):
    motifs= []

    for seq in Dna:
        i= random.randint(0, len(seq)-k)
        motifs.append(seq[i:i+k])

    return motifs

def RandomizedMotifSearch(Dna, k, t):
    # insert your code here
    motifs= RandomMotifs(Dna, k, t)
    BestMotifs= motifs

    while True:
        profile= ProfileWithPseudocounts(motifs)
        motifs= Motifs(profile, Dna)
        if Score(motifs) < Score(BestMotifs):
            BestMotifs= motifs
        else:
            return BestMotifs

def Normalize(Probabilities):
    # your code here
    normProb = {}
    for k,v in Probabilities.items():
        normProb[k] = Probabilities[k]/sum(Probabilities.values())
    return normProb

def WeightedDie(Probabilities):
    kmer = '' # output variable
    # your code here
    prob= Normalize(Probabilities)

    sumProb=0
    p= random.uniform(0,1)

    for k,v in prob.items():
        sumProb+= v
        if p < sumProb:
            kmer= k
            break

    return kmer

def ProfileGeneratedString(Text, profile, k):
    # your code here
    Prob= {}
    for i in range(len(Text)-k+1):
        Prob[Text[i:i+k]]= MotifPr(profile, Text[i:i+k])

    return WeightedDie(Prob)

def GibbsSampler(Dna, k, t, N):
    motifs= []
    for seq in Dna:
        i= random.randint(0, len(seq)-k)
        motifs.append(seq[i:i+k])

    BestMotifs = motifs # output variable

    # your code here
    for j in range(N):
        l= random.randint(0, len(Dna)-1)
        motifsNew= []

        for i in range(len(motifs)):
            if i != l:
                motifsNew.append(motifs[i])

        profile= ProfileWithPseudocounts(motifsNew)
        motifs[l]= ProfileGeneratedString(Dna[l], profile, k)

        if Score(motifs) < Score(BestMotifs):
            BestMotifs= motifs
    return BestMotifs

In [ ]:
DNA= ['ATGAGGTC', 'GCCCTAGA', 'AAATAGAT', 'TTGTGCTA']
motifs= ['GTC', 'CCC', 'ATA', 'GCT']
print(ProfileWithPseudocounts(motifs))
Motifs(ProfileWithPseudocounts(motifs), DNA)

{'A': [0.25, 0.125, 0.25], 'C': [0.25, 0.375, 0.375], 'G': [0.375, 0.125, 0.125], 'T': [0.125, 0.375, 0.25]}


['GTC', 'GCC', 'ATA', 'GCT']